In [4]:
import sqlite3
import pandas as pd

In [ ]:
schema_file_path = "schema.sql"

with open(schema_file_path, "r", encoding="utf-8") as f:
    schema_sql = f.read()
    
db_path = "sample_store.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.executescript(schema_sql)

conn.commit()
conn.close()

In [8]:
conn = sqlite3.connect("sample_store.db")

## Pull total number of orders that were completed on 18th March 2023 with the first name ‘John’ and last name Doe’

In [12]:
query1 = """
SELECT COUNT(*) AS total_orders
FROM SALES s
JOIN CUSTOMERS c ON s.Customer_id = c.Customer_id
WHERE s.Date = '2023-03-18'
  AND c.First_name = 'John'
  AND c.Last_name = 'Doe';
"""

pd.read_sql(query1, conn)


,total_orders
0,1


## Pull total number of customers that purchased in January 2023 and the average amount spend per customer

In [16]:
query2 = """
SELECT 
    COUNT(DISTINCT Customer_id) AS total_customers,
    AVG(total_spend) AS avg_spend_per_customer
FROM (
    SELECT Customer_id, SUM(Revenue) AS total_spend
    FROM SALES
    WHERE strftime('%Y-%m', Date) = '2023-01'
    GROUP BY Customer_id
);
"""

pd.read_sql(query2, conn)

,total_customers,avg_spend_per_customer
0,1,100.0


## Pull the departments that generated less than $600 in 2022

In [17]:
query3 = """
SELECT i.Department, SUM(s.Revenue) AS total_revenue
FROM SALES s
JOIN ITEMS i ON s.Item_id = i.Item_id
WHERE strftime('%Y', Date) = '2022'
GROUP BY i.Department
HAVING total_revenue < 600;
"""

pd.read_sql(query3, conn)

,Department,total_revenue
0,Clothing,270


## What is the most and least revenue we have generated by an order

In [18]:
query4 = """
SELECT 
    MAX(Revenue) AS max_order_revenue,
    MIN(Revenue) AS min_order_revenue
FROM SALES;
"""

pd.read_sql(query4, conn)

,max_order_revenue,min_order_revenue
0,3600,100


## What were the orders that were purchased in our most lucrative order

this is the result that ignore order with null id

In [23]:
query5 = """
WITH max_order AS (
    SELECT Order_id
    FROM SALES
    WHERE Order_id IS NOT NULL
    ORDER BY Revenue DESC
    LIMIT 1
)
SELECT s.*, c.First_name, c.Last_name, i.Item_name
FROM SALES s
JOIN CUSTOMERS c ON s.Customer_id = c.Customer_id
JOIN ITEMS i ON s.Item_id = i.Item_id
WHERE s.Order_id = (SELECT Order_id FROM max_order);
"""

pd.read_sql(query5, conn)

,Date,Order_id,Item_id,Customer_id,Quantity,Revenue,First_name,Last_name,Item_name
0,2024-03-01,1001,1,1,2,2400,John,Doe,Laptop


In [24]:
conn.close()